In [101]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import datetime as dt

In [102]:
loc = 'C:/Users/User/Desktop/time-travel/Stocks/'
os.chdir(loc)
filelist = os.listdir()

data = []
files = [x for x in filelist if x.endswith('.txt')] #  read files endswith .txt
files = [x for x in files if os.path.getsize(x) > 0] # remove file with size=0

In [103]:
for texts in files:
    df = pd.read_csv(texts)
    df['stock'] = texts.replace('.us.txt', '') #replace .us.txt with ''
    data.append(df)
data = pd.concat(data, ignore_index=True)
data.reset_index(inplace=True, drop=True) 

print(data.shape)

(14887665, 8)


In [104]:
data["Volume"] = data["Volume"]/10 # 10% , respect space time

In [105]:
data = data.drop(data.loc[(data.Open<=0) | (data.Low<=0)|(data.High<=0)|(data.Close<=0)|(data.Volume<1)].index) # clean data

In [106]:
data['Date'] = pd.to_datetime(data.Date)
data = data.sort_values(by='Date')
data['stock']  =  data['stock'].str.upper()
data.reset_index(inplace=True, drop=True)

In [107]:
print(data["OpenInt"].value_counts())
data = data.drop(columns=['OpenInt'])

0    14847108
Name: OpenInt, dtype: int64


In [108]:
data['stock'].nunique()

7163

In [109]:
#Set daily tranactions profit

Buy_open_sell_high = data['High']*0.99 - data['Open']*1.01
Buy_low_sell_close = data['Close']*0.99 - data['Low']*1.01
data['OP_HI'] = Buy_open_sell_high
data['LO_CL'] = Buy_low_sell_close
data['profit'] = data[['OP_HI', 'LO_CL']].max(axis=1)

In [110]:
#Remove outliers for High and Low

def is_outlier(s):
    lower_limit = s.mean() - (s.std() * 3)
    upper_limit = s.mean() + (s.std() * 3)
    return ~s.between(lower_limit, upper_limit)

data_clean = data[~data.groupby('stock')['High'].apply(is_outlier)]
data_clean = data[~data.groupby('stock')['Low'].apply(is_outlier)]

In [111]:
data_clean

,Date,Open,High,Low,Close,Volume,stock,OP_HI,LO_CL,profit
0,1962-01-02,0.6277,0.6362,0.6201,0.6201,257557.9,GE,-0.004139,-0.012402,-0.004139
1,1962-01-02,6.4130,6.4130,6.3378,6.3378,46705.6,IBM,-0.128260,-0.126756,-0.126756
2,1962-01-03,0.6201,0.6201,0.6122,0.6201,176474.9,GE,-0.012402,-0.004423,-0.004423
3,1962-01-03,6.3378,6.3963,6.3378,6.3963,35029.4,IBM,-0.068841,-0.068841,-0.068841
4,1962-01-04,0.6201,0.6201,0.6037,0.6122,219401.0,GE,-0.012402,-0.003659,-0.003659
...,...,...,...,...,...,...,...,...,...,...
14847103,2017-11-10,28.0300,28.4300,28.0300,28.1200,15406.1,GTY,-0.164600,-0.471500,-0.164600
14847104,2017-11-10,8.6384,10.4900,8.6100,10.4900,6652.0,GTXI,1.660316,1.689000,1.689000
14847105,2017-11-10,5.5690,5.5700,5.5200,5.5400,389.0,WILC,-0.110390,-0.090600,-0.090600
14847106,2017-11-10,1.6000,1.6800,1.5800,1.6300,11988.1,TWNKW,0.047200,0.017900,0.047200


In [112]:
High = data_clean.groupby(['stock']).agg({'High': ['mean', 'std']})
High=High.rename(columns={"mean": "high_mean", "std": "high_std"})

Low = data_clean.groupby(['stock']).agg({'Low': ['mean', 'std']})
Low=Low.rename(columns={"mean": "low_mean", "std": "low_std"})

In [113]:
Stocks=pd.concat([High, Low], axis=1)

In [114]:
Stocks

High                   Low           
       high_mean   high_std   low_mean    low_std
stock                                            
A      27.638731  11.980367  26.870007  11.701621
AA     24.726429  23.956769  24.032162  23.227892
AAAP   36.858587   9.360654  35.275316   9.055891
AABA   24.851730  15.682500  24.002056  15.159051
AAC    20.065225  10.194789  18.933637   9.670007
...          ...        ...        ...        ...
ZUMZ   23.951025   8.887726  22.947164   8.680691
ZVV    98.407500   3.465000  90.962500  11.704117
ZX      2.329210   1.236756   2.205517   1.166992
ZYME    9.234733   1.954402   8.811610   1.996126
ZYNE   13.992221   6.051420  12.862221   5.425969

[7151 rows x 4 columns]

In [115]:
Stocks.columns =Stocks.columns.droplevel()

In [116]:
# Set difference of means

Stocks["diff"]=Stocks["high_mean"]-Stocks["low_mean"]
Stocks=Stocks.sort_values(by=['diff'],ascending=False)

In [117]:
Stocks['Companies'] = Stocks.index

In [118]:
top100_extracted=Stocks['Companies'][:100]
top500_extracted=Stocks['Companies'][:500]

In [119]:
#https://www.fool.com/investing/2017/12/05/the-30-largest-companies-on-the-stock-market.aspx

top_comp = [
    "AAPL",  # Apple
    "GOOG", # Alphabet - Google
    "GOOGL",  # Alphabet - Google
    "MSFT",  # Microsoft
    "INTC",  # Intel
    "AMZN",  # Amazon
    "FB",  # Facebook
    "BRK-A", # Berkshire Hathaway
    "BRK-B", # Berkshire Hathaway
    "BABA", # Alibaba Group
    "JNJ", # Johnson & Johnson
    "JPM", # JPMorgan Chase & Co.
    "XOM", # ExxonMobil
    "BAC", # Bank of America
    "WMT", # Wal-Mart Stores Inc.
    "WFC", # Wells Fargo & Co.
    "RDS.A", # Royal Dutch Shell plc
    "RDS.B", # Royal Dutch Shell plc
    "V",  # Visa Inc.
    "PG", # Procter & Gamble Co.
    "BUD",  # Anheuser-Busch Inbev
    "T",  # AT&T Inc.
    "CVX",  # Chevron Corporation
    "UNH",  # UnitedHealth Group Inc.
    "PFE",  # Pfizer Inc.
    "RHHBY",  # Roche Holding Ltd.
    "CHL",  # China Mobile
    "HD",  # Home Depot
    "TSM",  # Taiwan Semiconductor
    "VZ",  # Verizon Communications
    "ORCL",  # Oracle Corporation
    "C",  # Citigroup Inc.
    "NVS",  # Novartis
]

In [120]:
Companies_small=np.concatenate((top100_extracted, top_comp))
Companies_large=np.concatenate((top500_extracted, top_comp))

In [121]:
Companies_small=np.unique(Companies_small)
Companies_large=np.unique(Companies_large)

In [122]:
print(len(Companies_small))
print(len(Companies_large))

130
526


In [123]:
final_small=data[data['stock'].isin(Companies_small)]
final_large=data[data['stock'].isin(Companies_large)]

In [124]:
final_small.to_csv('final_small.csv') 
final_large.to_csv('final_large.csv')

In [125]:
path='C:/Users/User/Desktop/time-travel/'

In [126]:
final_small.to_csv(os.path.join(path,r'final_small.csv'))
final_large.to_csv(os.path.join(path,r'final_large.csv'))